# Dataset: *putinmissing*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-src.csv', encoding = 'utf-8', header =0)
print(df.shape)
print(src_tw_df.shape)

(697, 11)
(238, 7)


In [4]:
df.isnull().sum()

src_tweet_id          0
src_user_id           0
src_tweet             0
src_date              0
reply_tweet_id      100
reply_user_id       100
reply_tweet         100
reply_date          100
label                 0
cleaned_reply_tw    125
reply_sentiment       0
dtype: int64

In [5]:
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)
df.shape

(572, 11)

In [6]:
src_tw_df.isnull().sum()

src_tweet_id      0
src_user_id       0
src_tweet         0
src_date          0
label             0
cleaned_src_tw    1
src_sentiment     0
dtype: int64

In [7]:
src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet

139    RT @L0gg0l: Rumors in Switzerland: #Putin abse...
Name: src_tweet, dtype: object

In [8]:
a = src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet_id.values
df.loc[df.src_tweet_id.isin(a),['reply_tweet','cleaned_reply_tw']]

,reply_tweet,cleaned_reply_tw


In [9]:
src_tw_df.dropna(inplace = True)
src_tw_df.reset_index(drop=True,inplace=True)
src_tw_df.shape

(237, 7)

## Emotion Analysis

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [10]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [11]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [12]:
emotion_ro = df.cleaned_reply_tw.apply(get_emotion_roBERT)

In [13]:
df['reply_emo_ro'] = emotion_ro

In [14]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-df-2.csv',index=False)

In [15]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [16]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\putinmissing-src-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [17]:
print("source tweet:", df.src_tweet[11],'\n')
print("original reply tw: ", df.reply_tweet[11])
print("cleaned reply tw: ", df.cleaned_reply_tw[11])

print('roBERT based: ',df.reply_emo_ro[11])


source tweet: I'm back! Few days of "me time". Secretly had a bit of work done. It's taken years off me!
#ПутинУмер
#putindead http://t.co/vDY5BykqVf 

original reply tw:  @JeSuisPutin  This isn't the "plastic surgery on the manboobs" that I had in mind; kudos!
cleaned reply tw:  isnt plastic surgery manboobs mind kudos
roBERT based:  joy
